This notebook requires:
* the North Sea test case from the [getm-setups repository](https://github.com/getm-model/setups)
* the [TPXO9 atlas](https://www.tpxo.net/global/tpxo9-atlas) to calculate tides at the open boundaries

In [ ]:
getm_setups_dir = '../../../getm-setups'

In [ ]:
import datetime
import os.path

import numpy

import pygetm
import pygetm.domain
import pygetm.input
import pygetm.legacy
import pygetm.input.tpxo

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_setups_dir, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=1, z0_const=0.001)
pygetm.legacy.load_bdyinfo(domain, os.path.join(getm_setups_dir, 'NorthSea/bdyinfo.dat'))
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)

fig, ax = matplotlib.pyplot.subplots()
ax.pcolormesh(domain.T.loni, domain.T.lati, numpy.ma.array(domain.T.H, mask=domain.T.mask==0));
shallow = (domain.T.H.ma < 5).filled(False)
ax.plot(domain.T.lon[shallow], domain.T.lat[shallow], 'xr')

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(12,12))
pc = ax.pcolormesh(domain.lon, domain.lat, domain.mask, shading='auto');
fig.colorbar(pc);
ax.plot(domain.T.lon[domain.bdy_j, domain.bdy_i], domain.T.lat[domain.bdy_j, domain.bdy_i], 'ow', mec='k', ms=2);
ax.set_title('mask on supergrid')

In [ ]:
bdy_lon = domain.T.lon[domain.bdy_j, domain.bdy_i]
bdy_lat = domain.T.lat[domain.bdy_j, domain.bdy_i]
tpxo = pygetm.input.tpxo.Dataset(bdy_lon, bdy_lat, verbose=True, root='../../../igotm/data/TPXO9')

In [ ]:
ref_date = datetime.datetime(2000, 1, 1)
fig, ax = matplotlib.pyplot.subplots()
z_masked = numpy.ma.array(domain.T.z, mask=domain.T.mask <= 0)
pc = ax.pcolormesh(domain.T.loni, domain.T.lati, z_masked, vmin=-2.5, vmax=2.5)
h = 0.001 * tpxo.update(ref_date)
sc = ax.scatter(bdy_lon, bdy_lat, c=h, edgecolors='k', vmin=-2.5, vmax=2.5, s=2, linewidth=.2)
cb = fig.colorbar(sc)
cb.set_label('elevation (m)')
title = ax.set_title(ref_date.strftime('%Y-%m-%d %H:%M'))

In [ ]:
# No surface forcing
tausx = domain.U.array(fill=0.0)
tausy = domain.V.array(fill=0.0)
sp = domain.T.array(fill=0.0)

timestep = 60.
timedelta = datetime.timedelta(seconds=timestep)
date = ref_date
for itime in range(60 *24 * 30):
    date += timedelta

    sim.zbdy[:] = 0.001 * tpxo.update(date)
    sim.update_sealevel_boundaries(timestep)

    sim.update_surface_pressure_gradient(domain.T.z, sp)
    sim.uv_momentum_2d(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.U.update_halos()
    sim.V.update_halos()
    sim.update_sealevel(timestep, sim.U, sim.V)
    sim.update_depth()

    if itime % 15 == 0:
        title.set_text(date.strftime('%Y-%m-%d %H:%M'))
        sc.set_array(sim.zbdy)
        pc.set_array(z_masked.ravel())

        fig.canvas.draw()